In [1]:
import numpy as np
from helper import *
from models import *
from cleaning import clean_tweets
import nltk
import pickle
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from nltk.corpus import stopwords
import string
from tqdm.notebook import tqdm

Reading twitter - 1grams ...
Reading twitter - 2grams ...


C:\Users\ahmed\anaconda3\lib\site-packages\ekphrasis\classes\exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
print('Indexing word vectors.')

embeddings_index = {}
DIM_SIZE = 25
f = open('../Data/glove.twitter.27B.25d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 1193515 word vectors.


In [4]:
with open('../Data/preprocessed_embeddings_index.pkl', 'wb') as f:
    pickle.dump(embeddings_index, f)

In [5]:
pos_tweets = []
neg_tweets = []
with open("../Data/twitter-datasets/train_pos_full.txt","r", encoding = 'utf8') as f:
    pos_tweets = f.readlines()
    f.close()
with open("../Data/twitter-datasets/train_neg_full.txt", "r", encoding = 'utf8') as f:
    neg_tweets = f.readlines()
    f.close()

In [6]:
text_processor = TextPreProcessor(
# terms that will be normalized
normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
    'time', 'url', 'date', 'number'],
# terms that will be annotated
annotate={"hashtag", "allcaps", "elongated", "repeated",
    'emphasis', 'censored'},
fix_html=True,  # fix HTML tokens

# corpus from which the word statistics are going to be used 
# for word segmentation 
segmenter="twitter", 

# corpus from which the word statistics are going to be used 
# for spell correction
corrector="twitter", 

unpack_hashtags=True,  # perform word segmentation on hashtags
unpack_contractions=True,  # Unpack contractions (can't -> can not)
spell_correct_elong=True,  # spell correction for elongated words

# select a tokenizer. You can use SocialTokenizer, or pass your own
# the tokenizer, should take as input a string and return a list of tokens
tokenizer=SocialTokenizer(lowercase=True).tokenize,

# list of dictionaries, for replacing tokens extracted from the text,
# with other expressions. You can pass more than one dictionaries.
dicts=[emoticons]
)

neg_tweets_cleaned = clean_tweets(neg_tweets,text_processor)
pos_tweets_cleaned = clean_tweets(pos_tweets,text_processor)

C:\Users\ahmed\anaconda3\lib\site-packages\ekphrasis\classes\tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


  0%|          | 0/1250000 [00:00<?, ?it/s]

  0%|          | 0/1250000 [00:00<?, ?it/s]

  0%|          | 0/1250000 [00:00<?, ?it/s]

  0%|          | 0/1250000 [00:00<?, ?it/s]

  0%|          | 0/1250000 [00:00<?, ?it/s]

  0%|          | 0/1250000 [00:00<?, ?it/s]

  0%|          | 0/1250000 [00:00<?, ?it/s]

  0%|          | 0/1250000 [00:00<?, ?it/s]

In [7]:
print(neg_tweets_cleaned[0])

['vinco', 'tresorpack', 'difficulty', 'object', 'disassemble', 'reassemble', 'wooden', 'pieces', 'beautiful', 'wo']


In [8]:
from models import *
training_data = []
for tweet in tqdm(neg_tweets_cleaned):
    emb = Tweet_to_GloVe(tweet,embeddings_index)
    training_data.append([np.array(emb), np.array([1.0, 0.0])])

for tweet in tqdm(pos_tweets_cleaned):
    emb = Tweet_to_GloVe(tweet,embeddings_index)
    training_data.append([np.array(emb),np.array([0.0, 1.0])])

print("Shuffiling and saving training_data")
np.random.shuffle(training_data)
np.save("training_data.npy", training_data)

  0%|          | 0/1250000 [00:00<?, ?it/s]

C:\Users\ahmed\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\ahmed\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


  0%|          | 0/1250000 [00:00<?, ?it/s]

Shuffiling and saving training_data


C:\Users\ahmed\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


(2500000, 2)


array([array([-0.417597  ,  0.22064498,  0.12767   , -0.331715  , -0.73833501,
              -0.3062    ,  1.72120005,  0.84535003, -0.73247002,  0.64841001,
              -0.99503499,  0.19454   , -3.72455001, -0.63418499,  0.62081499,
              -0.16795501,  0.90075499, -1.06484997, -1.1239    , -0.33067499,
               0.24281001, -0.02501   , -0.90321997,  1.45124996, -0.86731499]),
       array([0., 1.])], dtype=object)